<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/tensorboard%E5%8F%AF%E8%A7%86%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#标准开场,引入tf2.0
%tensorflow_version 2.x
import tensorflow as tf
print('tensorflow:',format(tf.__version__))
import numpy as np
from tensorflow import keras
import glob
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

tensorflow: 2.2.0-rc2


In [0]:
import datetime

In [0]:
#引入minist数据集
(train_image,train_label),(test_image,test_label)=tf.keras.datasets.mnist.load_data()

In [4]:
#给图像增加维度
train_image=tf.expand_dims(train_image,-1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [0]:
#test图像增加维度
test_image=tf.expand_dims(test_image,-1)

In [0]:
#预处理，改变数据类型
train_image=tf.cast(train_image/255,tf.float32)
test_image=tf.cast(test_image/255,tf.float32)

In [0]:
#预处理，转换label的数据类型
train_label=tf.cast(train_label,tf.int64)
test_label=tf.cast(test_label,tf.int64)

In [0]:
#把它作为tf.data加载进来,建立dataset
dataset=tf.data.Dataset.from_tensor_slices((train_image,train_label))

In [9]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [0]:
test_dataset=tf.data.Dataset.from_tensor_slices((test_image,test_label))

In [0]:
#对数据进行乱序，这里对10000个数据进行乱序混淆
#批次的大小,用batch
dataset=dataset.repeat().shuffle(10000).batch(32)
#这里没有设置repeat
test_dataset=test_dataset.batch(32)

In [0]:
#建立模型，进行自定义训练,把整个网络写在list里
#这里input_shape如何让添加任意形状的图片呢，我们可以这样写参数(none,none,1)
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(28,28,1)),
                           tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
                           tf.keras.layers.GlobalAveragePooling2D(),
                           tf.keras.layers.Dense(10)
                           ])
#这里最后一层我没有写激活函数所以我们再后面的lossfunction里有写from_logitc=ture

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##调用tensorboard

###区分不同时间所记录的tensorboard的记录值

In [0]:
import os
log_dir=os.path.join('logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
#为了实现对不同时间的log文件的记录

In [0]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir,
                                                    histogram_freq=1)

###使用回调函数

In [27]:
model.fit(dataset,
          epochs=5,
          steps_per_epoch=60000//128,
          validation_data=test_dataset,
          validation_steps=10000//128,
          callbacks=[tensorboard_callback])

Epoch 1/5
468/468 [==============================] - 2s 4ms/step - loss: 2.3026 - accuracy: 0.0903 - val_loss: 2.3026 - val_accuracy: 0.0990
Epoch 2/5
468/468 [==============================] - 2s 4ms/step - loss: 2.3026 - accuracy: 0.0931 - val_loss: 2.3026 - val_accuracy: 0.0990
Epoch 3/5
468/468 [==============================] - 2s 4ms/step - loss: 2.3026 - accuracy: 0.0929 - val_loss: 2.3026 - val_accuracy: 0.0990
Epoch 4/5
468/468 [==============================] - 2s 4ms/step - loss: 2.3026 - accuracy: 0.0966 - val_loss: 2.3026 - val_accuracy: 0.0990
Epoch 5/5
468/468 [==============================] - 2s 4ms/step - loss: 2.3026 - accuracy: 0.0945 - val_loss: 2.3026 - val_accuracy: 0.0990
